In [8]:
# load yaml
import yaml

p_yaml = 'results/without_rgb/mips-trained/google--flan-t5-base/version_42/hparams.yaml'
with open(p_yaml, 'r') as f:
    hparams = yaml.safe_load(f)

{'language_model_name': 'google/flan-t5-base', 'max_ctx_len': 256, 'head_name': 'af', 'lr': 1e-05, 'recall_ks': [1, 5, 10, 15, 20], 'train_backbone': False, 'caption_stride_sec': 2}


In [25]:
from sentence_transformers import CrossEncoder, SentenceTransformer
from transformers import BertForSequenceClassification
from IPython import display
print = display.display

model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2', max_length=512)
inputs = [
    ('Hello world', 'Hi there'),
    ('How are you?', 'I am fine'),
    ('This is a test', 'This is a test'),
    ('This is a test', 'This is a test'),
    ("How many people live in Berlin?", "Berlin had a population of 3,520,031 registered inhabitants in an area of 891.82 square kilometers."),
    ("How many people live in Berlin?", "Berlin is well known for its museums."),
]
model.predict(inputs)

array([-5.8606367, -8.096975 ,  6.014497 ,  8.6071415, -4.320079 ],
      dtype=float32)

In [26]:
list(zip(*inputs))

[('Hello world',
  'How are you?',
  'This is a test',
  'How many people live in Berlin?',
  'How many people live in Berlin?'),
 ('Hi there',
  'I am fine',
  'This is a test',
  'Berlin had a population of 3,520,031 registered inhabitants in an area of 891.82 square kilometers.',
  'Berlin is well known for its museums.')]

In [27]:
dict(model.tokenizer.__call__(*list(zip(*inputs)), return_tensors='pt', padding=True, truncation=True, add_special_tokens=False))

{'input_ids': tensor([[ 7592,  2088,  7632,  2045,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0],
         [ 2129,  2024,  2017,  1029,  1045,  2572,  2986,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0],
         [ 2023,  2003,  1037,  3231,  2023,  2003,  1037,  3231,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0],
         [ 2129,  2116,  2111,  2444,  1999,  4068,  1029,  4068,  2018,  1037,
           2313,  1997,  1017,  1010, 19611,  1010,  6021,  2487,  5068,  4864,
           1999,  2019,  2181,  1997,  6486,  2487,  

In [29]:
model.model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 384, padding_idx=0)
      (position_embeddings): Embedding(512, 384)
      (token_type_embeddings): Embedding(2, 384)
      (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-5): 6 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=384, out_features=384, bias=True)
              (key): Linear(in_features=384, out_features=384, bias=True)
              (value): Linear(in_features=384, out_features=384, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=384, out_features=384, bias=True)
              (LayerNorm): LayerNorm((384,), eps=1e-12, e

In [30]:
# model.smart_batching_collate_text_only(list(zip(*inputs)))
tokens = model.smart_batching_collate_text_only(inputs)
m: BertForSequenceClassification = model.model
print('원본')
print(dict(tokens))
print(m(**tokens))
print()

print('반전')
tokens['token_type_ids'] = 1 - tokens.token_type_ids
print(dict(tokens))
print(m.forward(**tokens))
print()

print('None')
tokens['token_type_ids'] = None
print(dict(tokens))
print(m.forward(**tokens))
print()

'원본'

{'input_ids': tensor([[  101,  7592,  2088,   102,  7632,  2045,   102,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0],
         [  101,  2129,  2024,  2017,  1029,   102,  1045,  2572,  2986,   102,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0],
         [  101,  2023,  2003,  1037,  3231,   102,  2023,  2003,  1037,  3231,
            102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0],
         [  101,  2129,  2116,  2111,  2444,  1999,  4068,  1029,   102,  4068,
           2018,  1037,  2313,  1997,  1017,  1010, 19611,  1010,  602

SequenceClassifierOutput(loss=None, logits=tensor([[-5.8606],
        [-8.0970],
        [ 6.0145],
        [ 8.6071],
        [-4.3201]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

'반전'

{'input_ids': tensor([[  101,  7592,  2088,   102,  7632,  2045,   102,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0],
         [  101,  2129,  2024,  2017,  1029,   102,  1045,  2572,  2986,   102,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0],
         [  101,  2023,  2003,  1037,  3231,   102,  2023,  2003,  1037,  3231,
            102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0],
         [  101,  2129,  2116,  2111,  2444,  1999,  4068,  1029,   102,  4068,
           2018,  1037,  2313,  1997,  1017,  1010, 19611,  1010,  602

SequenceClassifierOutput(loss=None, logits=tensor([[-8.7620],
        [-9.1768],
        [ 3.9660],
        [-0.4013],
        [-4.4644]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

'None'

{'input_ids': tensor([[  101,  7592,  2088,   102,  7632,  2045,   102,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0],
         [  101,  2129,  2024,  2017,  1029,   102,  1045,  2572,  2986,   102,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0],
         [  101,  2023,  2003,  1037,  3231,   102,  2023,  2003,  1037,  3231,
            102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0],
         [  101,  2129,  2116,  2111,  2444,  1999,  4068,  1029,   102,  4068,
           2018,  1037,  2313,  1997,  1017,  1010, 19611,  1010,  602

SequenceClassifierOutput(loss=None, logits=tensor([[-8.8202],
        [-8.4018],
        [-7.1589],
        [-3.0680],
        [-6.8644]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

# Time Embedding

In [11]:
from sentence_transformers import SentenceTransformer
from IPython.display import display

model = SentenceTransformer('multi-qa-mpnet-base-dot-v1')
inputs = sum([
    ('Hello world', 'Hi there'),
    ('How are you?', 'I am fine'),
    ('This is a test', 'This is a test')
], ())
model

You try to use a model that was created with version 3.0.0.dev0, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [2]:
tokens = model.tokenizer(inputs, padding='max_length', truncation=True, return_tensors='pt', add_special_tokens=False, max_length=12)
emask = model[0].auto_model.get_extended_attention_mask(tokens.attention_mask, tokens.input_ids.shape)

dict(tokens)

{'input_ids': tensor([[7596, 2092,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1],
         [7636, 2049,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1],
         [2133, 2028, 2021, 1033,    1,    1,    1,    1,    1,    1,    1,    1],
         [1049, 2576, 2990,    1,    1,    1,    1,    1,    1,    1,    1,    1],
         [2027, 2007, 1041, 3235,    1,    1,    1,    1,    1,    1,    1,    1],
         [2027, 2007, 1041, 3235,    1,    1,    1,    1,    1,    1,    1,    1]]),
 'attention_mask': tensor([[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}

In [3]:
from transformers.models.mpnet.modeling_mpnet import MPNetEmbeddings
type(model[0].auto_model.embeddings)

transformers.models.mpnet.modeling_mpnet.MPNetEmbeddings

In [4]:
z0 = model.forward(tokens).sentence_embedding
z0

tensor([[-0.5292, -0.2697, -0.2200,  ...,  0.0803,  0.0091, -0.2552],
        [-0.5293, -0.2697, -0.2200,  ...,  0.0803,  0.0091, -0.2552],
        [-0.2638, -0.5025, -0.2586,  ..., -0.0142, -0.0998, -0.3107],
        [-0.2711, -0.2618, -0.4834,  ..., -0.2342, -0.0891, -0.2003],
        [-0.1240, -0.5468, -0.2341,  ..., -0.0583, -0.0071, -0.0597],
        [-0.1240, -0.5468, -0.2341,  ..., -0.0583, -0.0071, -0.0597]],
       grad_fn=<CatBackward0>)

In [7]:
zz = model[0].auto_model.forward(**tokens, return_dict=True)
model[1].forward({'token_embeddings': zz.last_hidden_state, 'attention_mask': emask})['sentence_embedding']

tensor([[-0.5292, -0.2697, -0.2200,  ...,  0.0803,  0.0091, -0.2552],
        [-0.5293, -0.2697, -0.2200,  ...,  0.0803,  0.0091, -0.2552],
        [-0.2638, -0.5025, -0.2586,  ..., -0.0142, -0.0998, -0.3107],
        [-0.2711, -0.2618, -0.4834,  ..., -0.2342, -0.0891, -0.2003],
        [-0.1240, -0.5468, -0.2341,  ..., -0.0583, -0.0071, -0.0597],
        [-0.1240, -0.5468, -0.2341,  ..., -0.0583, -0.0071, -0.0597]],
       grad_fn=<CatBackward0>)

In [9]:
z = model[0].auto_model.embeddings.forward(**tokens)
z, = model[0].auto_model.encoder.forward(
	z, attention_mask=emask, head_mask=[None]*model[0].auto_model.config.num_hidden_layers)
z = model[1].forward({'token_embeddings': z, 'attention_mask': emask})['sentence_embedding']
# z = model[0].auto_model.pooler.forward(z)
z

tensor([[-0.5292, -0.2697, -0.2200,  ...,  0.0803,  0.0091, -0.2552],
        [-0.5293, -0.2697, -0.2200,  ...,  0.0803,  0.0091, -0.2552],
        [-0.2638, -0.5025, -0.2586,  ..., -0.0142, -0.0998, -0.3107],
        [-0.2711, -0.2618, -0.4834,  ..., -0.2342, -0.0891, -0.2003],
        [-0.1240, -0.5468, -0.2341,  ..., -0.0583, -0.0071, -0.0597],
        [-0.1240, -0.5468, -0.2341,  ..., -0.0583, -0.0071, -0.0597]],
       grad_fn=<CatBackward0>)

In [82]:
z0 / z

tensor([[ 7.4067e+01,  1.2836e+00, -1.4577e+00,  ..., -1.4139e+00,
          3.6675e-01, -1.5831e+00],
        [ 7.4035e+01,  1.2837e+00, -1.4578e+00,  ..., -1.4141e+00,
          3.6632e-01, -1.5830e+00],
        [ 2.6081e+00,  5.1416e+01,  1.6381e+00,  ...,  1.1244e-01,
         -7.8825e-01, -3.1044e+01],
        [ 2.1977e+00,  3.4790e+00, -6.5615e+01,  ..., -2.2938e+00,
         -7.1798e-01, -4.7595e+00],
        [ 5.9934e-01,  4.1159e+00, -2.1180e+00,  ...,  1.0424e+00,
         -4.4401e-02,  6.9190e-01],
        [ 5.9934e-01,  4.1159e+00, -2.1180e+00,  ...,  1.0424e+00,
         -4.4401e-02,  6.9190e-01]], grad_fn=<DivBackward0>)

In [32]:
list(model[0].modules())[1]

MPNetModel(
  (embeddings): MPNetEmbeddings(
    (word_embeddings): Embedding(30527, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): MPNetEncoder(
    (layer): ModuleList(
      (0-11): 12 x MPNetLayer(
        (attention): MPNetAttention(
          (attn): MPNetSelfAttention(
            (q): Linear(in_features=768, out_features=768, bias=True)
            (k): Linear(in_features=768, out_features=768, bias=True)
            (v): Linear(in_features=768, out_features=768, bias=True)
            (o): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (intermediate): MPNetIntermediate(
          (dense): Linear(in_